# Resample Fruad=1 Class
- Train using Keras DNN

In [9]:
import pandas as pd
import numpy as np
import keras

from collections import Counter

from sklearn.utils import resample, shuffle

from keras.layers import Dense, Dropout, BatchNormalization

from sklearn.metrics import roc_auc_score

from multiprocessing import cpu_count

from imblearn.under_sampling import NearMiss

import xgboost as xgb

!python -c 'import tensorflow as tf; print(tf.version)'
print(cpu_count())

<module 'tensorflow._api.v1.version' from '/home/ec2-user/anaconda3/lib/python3.7/site-packages/tensorflow/_api/v1/version/__init__.py'>
2


# Read Dataset

In [2]:
def get_string_features(df):
    string_features = []
    for col in df.columns:
        if df[col].dtype == np.dtype('object'):
            string_features.append(col)

    return string_features


def read_data(drop_string_features=True):
    
    df = pd.read_csv('./datasets/kfold/transaction_fold_0_0_0.csv')
    
    if drop_string_features:
        string_features = get_string_features(df)
        df = df.drop(columns=string_features)
    
    return df
    
    
train = read_data(drop_string_features=True)

In [3]:
train.shape

(73816, 380)

In [4]:
train.isFraud.value_counts()

0    71376
1     2440
Name: isFraud, dtype: int64

# Undersample majority class

In [5]:
def preprocess(df):
    df = df.fillna(-999)
    
    return df.drop(columns=['TransactionID', 'TransactionDT'])


train = preprocess(train)

In [6]:
train.shape

(73816, 378)

In [7]:
def undersample(df, random_state=27):
    """
    Upsample minority class (isFraud=1), combine with majority class, and then shuffle them.
    """
    
    # Declare Random Under Sampler
    rus = NearMiss(version=3, 
                   n_jobs=cpu_count(),
                   random_state=random_state)
    
    return rus.fit_resample(df.drop(columns=['isFraud']),
                           df['isFraud'])


%time X, y = undersample(train)

y = y.astype('int32')

CPU times: user 31.2 s, sys: 633 ms, total: 31.8 s
Wall time: 24.3 s


In [8]:
print(X.shape)
print(y.shape)

(4880, 377)
(4880,)


In [10]:
Counter(y)

Counter({0: 2440, 1: 2440})

# Build Model

In [14]:
def get_model(input_dim, lr=1e-3):
    
#     inputs = keras.layers.Input(shape=[input_dim,])
    
#     x = keras.layers.Dense(256, activation='relu')(inputs)
#     x = keras.layers.Dense(128, activation='relu')(x)
#     x = keras.layers.Dense(64, activation='relu')(x)
#     x = keras.layers.Dense(16, activation='relu')(x)
#     outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    
#     model = keras.models.Model(inputs=inputs,
#                               outputs=outputs)
    

    model = keras.models.Sequential()
    model.add(Dense(256, activation='relu', kernel_initializer = 'uniform', input_dim=input_dim))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid', kernel_initializer = 'uniform'))
    
    optimizer = keras.optimizers.Adam(lr=lr)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                 metrics=['accuracy'])
    
    
    return model


model = get_model(X.shape[1], lr=1e-3)

model.summary()

W0802 07:11:11.247573 140676939990720 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 07:11:11.269109 140676939990720 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 07:11:11.273927 140676939990720 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 07:11:11.363242 140676939990720 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.p

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               96768     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
__________

# Train

In [15]:
epochs = 50
batch_size = 256

model.fit(X, y,
         epochs=epochs,
         batch_size=batch_size)

Epoch 1/50
4880/4880 [==============================] - 2s 403us/step - loss: 0.6917 - acc: 0.5277
Epoch 2/50
4880/4880 [==============================] - 0s 50us/step - loss: 0.6870 - acc: 0.5570
Epoch 3/50
4880/4880 [==============================] - 0s 48us/step - loss: 0.6792 - acc: 0.5701
Epoch 4/50
4880/4880 [==============================] - 0s 49us/step - loss: 0.6740 - acc: 0.5691
Epoch 5/50
4880/4880 [==============================] - 0s 48us/step - loss: 0.6630 - acc: 0.6018
Epoch 6/50
4880/4880 [==============================] - 0s 49us/step - loss: 0.6614 - acc: 0.6018
Epoch 7/50
4880/4880 [==============================] - 0s 48us/step - loss: 0.6536 - acc: 0.6180
Epoch 8/50
4880/4880 [==============================] - 0s 48us/step - loss: 0.6471 - acc: 0.6244
Epoch 9/50
4880/4880 [==============================] - 0s 49us/step - loss: 0.6444 - acc: 0.6254
Epoch 10/50
4880/4880 [==============================] - 0s 49us/step - loss: 0.6431 - acc: 0.6344
Epoch 11/50
4880/4

In [16]:
pred_prob = model.predict_proba(X,
                               batch_size=batch_size)

score = roc_auc_score(y, pred_prob)

print('roc-auc score={}'.format(score))


roc-auc score=0.7673652076054824


# Train using XGB

In [17]:
seed = 27
model = xgb.XGBClassifier(objective='binary:logistic',
                            n_thread=cpu_count(),
                            seed=seed)

%time model.fit(X, y, verbose=True)

CPU times: user 3.44 s, sys: 7.5 ms, total: 3.45 s
Wall time: 3.48 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              n_thread=2, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=27, silent=None, subsample=1, verbosity=1)

In [18]:
pred_prob = model.predict_proba(X)

score = roc_auc_score(y, pred_prob[:, 1])

print('roc-auc score={}'.format(score))

roc-auc score=0.8989602089492073
